In [66]:
import pandas as pd
import pyarrow.parquet as pq
import datetime

In [67]:
df = pq.read_table(source="/home/toliman/Desktop/257-2021-4-player-log.parquet").to_pandas()

In [68]:
print(df.shape)
new_df = pd.DataFrame(columns=('month', 'week_day', 'half_hour', 'efficiency', 'unix', 'views'), )

(516739, 14)


In [69]:
for i in range(df.shape[0]):
    if (i % 1000 == 0):
        print(i // 1000)
    obj = df.loc[i]
    in_t = datetime.datetime.fromtimestamp(obj['DateTimeInTick'] / 1000)
    out_t = datetime.datetime.fromtimestamp(obj['DateTimeOutTick'] / 1000)
    month_1 = in_t.month
    month_2 = out_t.month
    weekday_1 = in_t.weekday()
    weekday_2 = out_t.weekday()
    halfhour_1 = 2 * in_t.hour + 1 + int(in_t.minute >= 30)
    halfhour_2 = 2 * in_t.hour + 1 + int(in_t.minute >= 30)
    efficiency = obj['MacCountAll'] / obj['Duration']

    new_df = new_df.append({'month': month_1, 'week_day': weekday_1, 'half_hour': halfhour_1, 'efficiency': efficiency,
                            'unix': obj['DateTimeOutTick'], 'views': int(obj['MacCountAll'])},
                           ignore_index=True)

    if month_1 != month_2 or weekday_1 != weekday_2 or halfhour_1 != halfhour_2:
        new_df = new_df.append(
            {'month': month_2, 'week_day': weekday_2, 'half_hour': halfhour_2, 'efficiency': efficiency,
             'unix': obj['DateTimeOutTick'], 'views': int(obj['MacCountAll'])},
            ignore_index=True)


0


In [70]:
new_df.shape

(500, 6)

In [71]:
new_df

,month,week_day,half_hour,efficiency,unix,views
0,4,3,10,2.4,1617242398103,12
1,4,3,10,2.2,1617242393097,11
2,4,3,10,2.8,1617242388097,14
3,4,3,10,2.6,1617242383087,13
4,4,3,10,3.4,1617242378073,17
...,...,...,...,...,...,...
495,4,3,9,2.6,1617240149763,13
496,4,3,9,2.6,1617240144760,13
497,4,3,9,1.8,1617240139750,9
498,4,3,9,2,1617240134737,10


In [72]:
new_df.to_csv('/home/toliman/Desktop/preprocessing_data.csv')
